## Investigates the gains for different entropy-coding schemes

In [1]:
import bench_utils
from algos.alg_fpd_extended import FpdExtended
from shapely.geometry import shape
from algos.fpd_extended_lib.compress import get_zz_encoded_delta
from algos.fpd_extended_lib.decompress import decode_header
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
fpde = FpdExtended()
#%pip install dill
import dill as pickle
import math
from intersection.plotting import xs, ys
import numpy as np

ENTROPY_BASE_PATH = "data/entropy_models"

In [2]:
# Load a bunch of data-sets
# chunks[delta_size] - contains a list of chunks, first indexed by the number of bits used per delta


from collections import defaultdict
from bitarray import bitarray
from algos.fpd_extended_lib import cfg
import shapely
import tqdm

shape_cnt = 0
dt_to_coord = defaultdict()
dt_freq = defaultdict(lambda: defaultdict(int))
dt_xs = []
dt_ys = []
chunks = defaultdict(list)
failed = 0

def append_deltas(dataset, is_shp_dir=False):
    global shape_cnt
    global failed
    if not is_shp_dir:
        df, idxs = bench_utils.read_dataset(dataset)
    else:
        df, idxs = bench_utils.load_shp_files(dataset)

    for idx in tqdm.tqdm(idxs):
        shape_cnt += 1
        geom = shape(df.iloc[idx]) if not is_shp_dir else df.iloc[idx].geometry
        if geom.geom_type not in ["LineString", "Polygon", "MultiPolygon"]:
            continue
        try: 
            bin = fpde.compress(geom)[1]
            cfg.offset = 0
            delta_size, type = decode_header(bin)
            chks = fpde.get_chunks(bin, include_ring_start=False)[0]
            # Go though deltas
            for chk in chks:
                chk_dts = []
                for i in range(len(chk) - 1):
                    dt_x = get_zz_encoded_delta(chk[i][0], chk[i + 1][0])
                    dt_y = get_zz_encoded_delta(chk[i][1], chk[i + 1][1])
                    dt_to_coord[dt_x] = chk[i + 1][0] - chk[i][0]
                    dt_to_coord[dt_y] = chk[i + 1][1] - chk[i][1]
                    dt_freq[delta_size][dt_x] += 1
                    dt_freq[delta_size][dt_y] += 1
                    dt_xs.append(dt_x)
                    dt_ys.append(dt_y)
                    chk_dts += [dt_x, dt_y]
                chunks[delta_size].append(chk_dts)
        except:
            print("Failed one geom!")
            #print(geom)
            #return
            failed += 1
                
GENERATE_DATA = False
SAVE_DATA = False
GENERATE_MODELS = False # Set to False to load stored models
SAVE_MODELS = False

if GENERATE_DATA:
    append_deltas("data/sweden-latest-free", is_shp_dir=True)
    append_deltas("data/lund_building_highway.json")
    append_deltas("data/world_7_dec.json")
    append_deltas("data/sweden.json")

    if SAVE_DATA:
        with open('data/delta_freqs.pkl', 'wb') as f:
            pickle.dump([shape_cnt, dt_to_coord, dt_freq, dt_xs, dt_ys, chunks], f)
    print("Failed:", failed)
else:
    with open('data/delta_freqs.pkl', 'rb') as f:
        shape_cnt, dt_to_coord, dt_freq, dt_xs, dt_ys, chunks = pickle.load(f)
    

print(f"Number of shapes: {shape_cnt}")
print(f"Number of deltas: {len(dt_xs) + len(dt_ys)}")

Number of shapes: 7283780
Number of deltas: 186669896


In [3]:
dt_freq_sorted = defaultdict(lambda: defaultdict(int))
dt_freq_sorted_key = defaultdict(lambda: defaultdict(int))
for bit_size in sorted(dt_freq.keys()):
    freqs = sorted(dt_freq[bit_size].items(), key=lambda x: -x[1])
    dt_freq_sorted[bit_size] = freqs
    dt_freq_sorted_key[bit_size] = sorted(dt_freq[bit_size].items(), key=lambda x: x[0])

DISP = 12
print(dt_freq_sorted[DISP])
print(dt_freq_sorted_key[DISP])


# merged_dict = {}
# for _, dictionary in dt_freq_sorted.items():
#     for key, value in dictionary:
#         if key in merged_dict:
#             merged_dict[key] += value
#         else:
#            merged_dict[key] = value

# tot_sum = 0
# for key, value in sorted(merged_dict.items(), key=lambda x: -x[1])[:10]:
#     tot_sum += 100 * value / sum(merged_dict.values())
# print(tot_sum)

# tot_sum = 0
# for key, value in sorted(merged_dict.items(), key=lambda x: -x[1])[10:100]:
#     tot_sum += 100 * value / sum(merged_dict.values())
# print(tot_sum)

6.105669550488205
8.077080087943052


In [5]:
#sns.kdeplot()

HEIGHT = 2
WIDTH = 3

PLOT = False

if PLOT:
    tot_dts = len(dt_xs) + len(dt_ys)
    fig, axes = plt.subplots(HEIGHT, WIDTH)
    cnt_plotted = 0
    for idx, bit_size in enumerate(sorted(dt_freq.keys())):
        chks = chunks[bit_size]
        dts = [item for sublist in chks for item in sublist]
        if len(dts)/tot_dts > 0.01 and cnt_plotted < 6:
            curr_height, curr_width = cnt_plotted // WIDTH, cnt_plotted % WIDTH
            current_axis = axes[curr_height, curr_width]
            current_axis.set_title(f"{bit_size} bits/delta ({round(100 * len(dts)/tot_dts, 1)}%)", fontsize=10)

            #data = list(dt_freq[bit_size].items())

            #ax = sns.kdeplot(dts, ax=current_axis)
            #ax = sns.scatterplot(x = xs(data), y=ys(data), ax=current_axis, size=0.1)
            ax = sns.histplot(dts, ax=current_axis, bins=80, kde=True)
            ax.ticklabel_format(style='plain', axis='y')
            current_axis.set_xlim(0, max(dts))
            current_axis.set(ylabel=None)
            cnt_plotted += 1
            

    fig.supylabel('Count')
    fig.supxlabel('Delta Values')
    fig.suptitle('Deltas for Different Bit-Lengths')
    fig.set_figwidth(WIDTH * 3)
    fig.set_figheight(HEIGHT * 3)
    plt.ticklabel_format(style='plain', axis='y')
    plt.tight_layout()
    plt.show()

In [6]:
# for i in range(1):
#     idx = random.randint(0, len(deltas_fp))
#     print("---")
#     print("Delta in FP and LONG:    ", f'{deltas_fp[idx]:.14f}', '        ', deltas_long[idx])
#     print("Converting to f32:       ", f'{struct.unpack("!f", struct.pack("!f", deltas_fp[idx]))[0]:.14f}')
#     print("Required bits:           ", math.ceil(math.log2(deltas_long[idx] + 1)))
#     bin = util.int2ba(deltas_long[idx], length=64)
#     print("ZZ-encoded:              ", end=None)
#     util.pprint(bin, width=200)

#     bin = bitarray()
#     print("FP directly to bin:      ", end=None)
#     bin.frombytes(struct.pack("!d", deltas_fp[idx]))
#     util.pprint(bin, width=200)

#     bin = bitarray()
#     print("FP32 directly to bin:    ", end=None)
#     bin.frombytes(struct.pack("!f", deltas_fp[idx]))
    
#     util.pprint(bin, width=200)

## Calculate Entropy

In [7]:
total_freq = defaultdict(int)
sum_none_size = 0
sum_opt_size = 0
tot_d_cnt = 0

for idx, bit_size in enumerate(sorted(dt_freq.keys())):
    entropy = 0
    chks = chunks[bit_size]
    freq = dt_freq[bit_size]
    tot = sum(freq.values())
    tot_d_cnt += tot
    sum_none_size += bit_size * tot
    for i in freq:
        total_freq[i] += freq[i]
        p_i = freq[i] / tot
        entropy += - p_i * np.log2(p_i)
    print(bit_size, ": Bits/delta", entropy)
    sum_opt_size += entropy * tot
#road building, one more
entropy = 0
tot = sum(total_freq.values())
for i in total_freq:
    p_i = total_freq[i] / tot
    entropy += - p_i * np.log2(p_i)

none_bit_len = sum_none_size/tot_d_cnt
per_delta_bit_len = sum_opt_size/tot_d_cnt
print("TOT: Bits/delta (one model):", entropy, "(%, factor):", entropy/none_bit_len, none_bit_len/entropy)
print("TOT: Bits/delta (model per delta):", per_delta_bit_len, "(%, factor):", per_delta_bit_len/none_bit_len, none_bit_len/per_delta_bit_len)
print("TOT: Average bits/delta no entropy:", none_bit_len)

2 : Bits/delta 1.0
3 : Bits/delta 2.8241690761232876
4 : Bits/delta 3.8528015921970797
5 : Bits/delta 4.7761342341451805
6 : Bits/delta 5.834841626385016
7 : Bits/delta 6.763260964766167
8 : Bits/delta 7.753372352345719
9 : Bits/delta 8.70573018084362
10 : Bits/delta 9.796967594601513
11 : Bits/delta 10.81098386575361
12 : Bits/delta 11.52658882613758
13 : Bits/delta 12.066797312006731
14 : Bits/delta 12.057649347412971
15 : Bits/delta 13.129832606099459
16 : Bits/delta 14.294805159654274
17 : Bits/delta 15.115839471343914
18 : Bits/delta 15.614314473377364
19 : Bits/delta 13.26743958581936
20 : Bits/delta 12.842860027901164
21 : Bits/delta 13.323355505646141
22 : Bits/delta 13.553094185715523
23 : Bits/delta 12.6861006324161
24 : Bits/delta 8.680618827332802
TOT: Bits/delta (one model): 12.93601926284609 (%, factor): 0.9173626542492038 1.0900814365703921
TOT: Bits/delta (model per delta): 12.512843837832728 (%, factor): 0.8873530103846289 1.126947210745973
TOT: Average bits/delta no e

## Encoding Huffman

In [8]:
def evaluate_encoder(setup, bit_sizes=[]):
    glob_size_coded = 0
    glob_size_normal = 0
    keys = list(filter(lambda x: x != 0, sorted(dt_freq.keys())))
    if len(bit_sizes) != 0:
        keys = list(filter(lambda x: x in bit_sizes, keys))
    for idx, bit_size in enumerate(keys):
        chks = chunks[bit_size]
        dts = [item for sublist in chks for item in sublist]
        dt_freqs = dict(dt_freq_sorted[bit_size])
        get_encoded_size = setup(dt_freqs, chks, dts, bit_size)
        
        size_coded = 0
        size_normal = 0
        for chk in tqdm.tqdm(chks):
            size_coded += get_encoded_size(chk)
            size_normal += len(chk) * bit_size

        print(f"---: {bit_size} : DELTAS {len(dts)} ({len(dts) / len(dt_xs + dt_ys)}) :---")
        print("Deltas present:", len(dt_freqs.keys()) / math.pow(2, bit_size))
        print("Enc:", size_coded)
        print("Nor:", size_normal)
        print("Percentage:", size_coded / size_normal)
        print("Factor:", size_normal / size_coded)
        print("Bit-size:", bit_size * (size_coded / size_normal))
        glob_size_coded += size_coded
        glob_size_normal += size_normal


    print(f"\n---: IN TOTAL :---")
    print("Enc:", glob_size_coded)
    print("Nor:", glob_size_normal)
    print("Percentage:", glob_size_coded / glob_size_normal)
    print("Factor:", glob_size_normal / glob_size_coded)
    print("Bit-size:", glob_size_coded / tot_d_cnt)
    print("Bit-size no coding:", glob_size_normal / tot_d_cnt)

In [9]:
#%pip install dahuffman
from dahuffman import HuffmanCodec
from bitarray import util

'''
    Huffman encoding of deltas. Note that this tree does not support missing deltas, hence the implementation
    below this cell is used in FPDE.
'''
def huff_setup(dt_freqs, chks, dts, bit_size):

    if GENERATE_MODELS:
        codec = util.huffman_code(dt_freqs)
        if SAVE_MODELS:
            with open(f'{ENTROPY_BASE_PATH}/huff_{bit_size}.pkl', 'wb') as f:
                pickle.dump([bit_size, codec], f)
    else:
        with open(f'{ENTROPY_BASE_PATH}/huff_{bit_size}.pkl', 'rb') as f:
            _, codec = pickle.load(f)

    def huff(chk):
        encoded = bitarray()
        encoded.encode(codec, chk)
        USE_TOGGLE_BIT = False
        if USE_TOGGLE_BIT:
            min_size = min(len(encoded), len(chk) * bit_size)
            size = min_size + 1
        else:
            size = len(encoded)
        return size
    return huff

evaluate_encoder(huff_setup)

  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: symbol not defined in prefix code: 0

# Huffman for common values, else encode whole delta

In [ ]:
#%pip install dahuffman
from dahuffman import HuffmanCodec
from bitarray import util

def huff_setup(dt_freqs, chks, dts, bit_size):

    if GENERATE_MODELS:
        N = len(dt_freqs.keys())
        dt_freqs_stripped = dict(filter(lambda x: x[1] > 3, dt_freqs.items()))
        # Append outlier symbol
        sum_values = sum(list(dt_freqs_stripped.values()))
        dt_freqs_stripped[-1] = sum_values * (1 - len(dt_freqs_stripped) / N) # Estimate probablity based on outliers above
        codec = util.huffman_code(dt_freqs_stripped)
        if SAVE_MODELS:
            with open(f'{ENTROPY_BASE_PATH}/huff_{bit_size}.pkl', 'wb') as f:
                pickle.dump([bit_size, codec], f)
    else:
        with open(f'{ENTROPY_BASE_PATH}/huff_{bit_size}.pkl', 'rb') as f:
            _, codec = pickle.load(f)

    def huff(chk):
        encoded = bitarray()
        chk_valid = [x if x in codec.keys() else -1 for x in chk]
        outliers = chk_valid.count(-1)
        encoded.encode(codec, chk_valid)
        size = len(encoded)
        return size + outliers * bit_size
    return huff

evaluate_encoder(huff_setup, bit_sizes=[])

100%|██████████| 1/1 [00:00<00:00, 2991.66it/s]


---: 2 : DELTAS 2 (1.0714100360349481e-08) :---
Deltas present: 0.5
Enc: 6
Nor: 4
Percentage: 1.5
Factor: 0.6666666666666666
Bit-size: 3.0


100%|██████████| 26/26 [00:00<00:00, 52835.22it/s]


---: 3 : DELTAS 708 (3.792791527563716e-06) :---
Deltas present: 1.0
Enc: 2085
Nor: 2124
Percentage: 0.981638418079096
Factor: 1.018705035971223
Bit-size: 2.944915254237288


100%|██████████| 14/14 [00:00<00:00, 36838.30it/s]


...

---: 22 : DELTAS 135286 (0.0007247338906751199) :---
Deltas present: 0.007027864456176758
Enc: 2104903
Nor: 2976292
Percentage: 0.7072232831993635
Factor: 1.4139805967305856
Bit-size: 15.558912230385996


100%|██████████| 721/721 [00:00<00:00, 108944.92it/s]


---: 23 : DELTAS 22982 (0.00012311572724077588) :---
Deltas present: 0.0013194084167480469
Enc: 421156
Nor: 528586
Percentage: 0.7967596568959451
Factor: 1.2550836269695789
Bit-size: 18.325472108606736


100%|██████████| 25/25 [00:00<00:00, 74367.09it/s]


---: 24 : DELTAS 1200 (6.428460216209688e-06) :---
Deltas present: 4.589557647705078e-05
Enc: 22839
Nor: 28800
Percentage: 0.7930208333333333
Factor: 1.261000919479837
Bit-size: 19.0325

---: IN TOTAL :---
Enc: 2345465686
Nor: 2632290904
Percentage: 0.8910358966920626
Factor: 1.122289240772973
Bit-size: 12.56477737577997
Bit-size no coding: 14.10131446154553


In [ ]:
#%pip install golomb_coding
from golomb_coding import golomb_coding, optimal_golomb_coding
from bitarray import util

golomb_param = {}

def golomb_setup(dt_freqs, chks, dts, bit_size):
    delta_mean = sum(dts) / len(dts)
    golden_ratio = (math.sqrt(5) + 1) / 2
    param = max(0, 1 + math.floor(math.log2(math.log(golden_ratio - 1) / math.log(delta_mean / (delta_mean + 1)))))
    param = int(math.pow(2, param))
    golomb_param[bit_size] = param
    tot = sum(dt_freqs.values())
    def golomb(chk):
        encoded = [golomb_coding(d, param) for d in chk]
        #encoded = [optimal_golomb_coding(d, dt_freqs[d] / tot) for d in chk]
        return sum(list(map(lambda x: len(x), encoded)))
    return golomb

evaluate_encoder(golomb_setup, bit_sizes=[])

100%|██████████| 1/1 [00:00<00:00, 5645.09it/s]

---: 2 : DELTAS 2 (1.0714100360349481e-08) :---
Deltas present: 0.5
Enc: 5
Nor: 4
Percentage: 1.25
Factor: 0.8
Bit-size: 2.5



100%|██████████| 26/26 [00:00<00:00, 12936.17it/s]


---: 3 : DELTAS 708 (3.792791527563716e-06) :---
Deltas present: 1.0
Enc: 2236
Nor: 2124
Percentage: 1.0527306967984935
Factor: 0.9499105545617174
Bit-size: 3.1581920903954805


100%|██████████| 14/14 [00:00<00:00, 11493.49it/s]


...

---: 22 : DELTAS 135286 (0.0007247338906751199) :---
Deltas present: 0.007027864456176758
Enc: 2803545
Nor: 2976292
Percentage: 0.9419589878950049
Factor: 1.061617345182617
Bit-size: 20.723097733690107


100%|██████████| 721/721 [00:00<00:00, 13627.01it/s]


---: 23 : DELTAS 22982 (0.00012311572724077588) :---
Deltas present: 0.0013194084167480469
Enc: 488659
Nor: 528586
Percentage: 0.9244645147620255
Factor: 1.0817072846299771
Bit-size: 21.262683839526588


100%|██████████| 25/25 [00:00<00:00, 8237.05it/s]


---: 24 : DELTAS 1200 (6.428460216209688e-06) :---
Deltas present: 4.589557647705078e-05
Enc: 27375
Nor: 28800
Percentage: 0.9505208333333334
Factor: 1.0520547945205478
Bit-size: 22.8125

---: IN TOTAL :---
Enc: 2488048783
Nor: 2632290904
Percentage: 0.9452028190422224
Factor: 1.0579739923049571
Bit-size: 13.328602181253693
Bit-size no coding: 14.10131446154553


In [ ]:
#inverted_unary(n // b)+minimal_binary_coding(n % b, b)

from minimal_binary_coding import minimal_binary_coding
from unary_coding import inverted_unary


def golomb_huff_setup(dt_freqs, chks, dts, bit_size):
    delta_mean = sum(dts) / len(dts)
    golden_ratio = (math.sqrt(5) + 1) / 2
    param = max(0, 1 + math.floor(math.log2(math.log(golden_ratio - 1) / math.log(delta_mean / (delta_mean + 1)))))
    param = int(math.pow(2, param))
    k_freqs = defaultdict(int)

    #k_freq_dist = []
    for d, cnt in dt_freqs.items():
        k = inverted_unary(d // param)
        k_freqs[k] += cnt
        #k_freq_dist += [k for _ in range(cnt)]
    
    #sns.histplot(k_freq_dist, bins=40)
    k_codec = util.huffman_code(k_freqs)

    def golomb_huff(chk):
        enc_len = 0
        for d in chk:
            k = inverted_unary(d // param)
            r = minimal_binary_coding(d % param, param)
            enc_len += len(k_codec[k].to01()) + len(r)

        return enc_len
    return golomb_huff

evaluate_encoder(golomb_huff_setup, bit_sizes=[])

100%|██████████| 1/1 [00:00<00:00, 12018.06it/s]


---: 2 : DELTAS 2 (1.0714100360349481e-08) :---
Deltas present: 0.5
Enc: 2
Nor: 4
Percentage: 0.5
Factor: 2.0
Bit-size: 1.0


100%|██████████| 26/26 [00:00<00:00, 12777.02it/s]

---: 3 : DELTAS 708 (3.792791527563716e-06) :---
Deltas present: 1.0
Enc: 2124
Nor: 2124
Percentage: 1.0
Factor: 1.0
Bit-size: 3.0



100%|██████████| 14/14 [00:00<00:00, 9702.62it/s]


...

---: 22 : DELTAS 135286 (0.0007247338906751199) :---
Deltas present: 0.007027864456176758
Enc: 2803541
Nor: 2976292
Percentage: 0.9419576439408499
Factor: 1.0616188598632943
Bit-size: 20.7230681666987


100%|██████████| 721/721 [00:00<00:00, 13934.18it/s]


---: 23 : DELTAS 22982 (0.00012311572724077588) :---
Deltas present: 0.0013194084167480469
Enc: 484313
Nor: 528586
Percentage: 0.9162425792586258
Factor: 1.0914140235756629
Bit-size: 21.073579322948394


100%|██████████| 25/25 [00:00<00:00, 7339.89it/s]


---: 24 : DELTAS 1200 (6.428460216209688e-06) :---
Deltas present: 4.589557647705078e-05
Enc: 27369
Nor: 28800
Percentage: 0.9503125
Factor: 1.0522854324235449
Bit-size: 22.8075

---: IN TOTAL :---
Enc: 2484531327
Nor: 2632290904
Percentage: 0.9438665472818881
Factor: 1.0594718107975782
Bit-size: 13.309758992955135
Bit-size no coding: 14.10131446154553


In [ ]:
from bitarray import bitarray, util

def fpzip_setup(dt_freqs, chks, dts, bit_size):
    k_freqs = defaultdict(int)
    r_freqs = defaultdict(int)
    for d, cnt in dt_freqs.items():
        k = int(math.floor(np.log2(d))) if d != 0 else -1
        k_freqs[k] += cnt
        #if k != -1:
            #r = d - int(math.pow(2, k))
            #r_freqs[r] += 1
    #k_codec = HuffmanCodec.from_frequencies(k_freqs)
    #r_codec = HuffmanCodec.from_frequencies(r_freqs)
    k_codec = util.huffman_code(k_freqs)
    #r_codec = util.huffman_code(r_freqs)

    def fpzip(chk):
        size = 0
        for d in chk:
            k = int(math.floor(np.log2(d))) if d != 0 else -1
            r = d - int(math.pow(2, k))
            #size += len(k_codec.encode([k])) + (len(r_codec.encode([r])) if k != -1 else 0)
            #size += len(k_codec[k].to01()) + (len(r_codec[r].to01()) if k != -1 else 0)
            size += len(k_codec[k].to01()) + k if k != -1 else 0
        return size 
    return fpzip

evaluate_encoder(fpzip_setup)

100%|██████████| 1/1 [00:00<00:00, 3609.56it/s]


---: 2 : DELTAS 2 (1.0714100360349481e-08) :---
Deltas present: 0.5
Enc: 2
Nor: 4
Percentage: 0.5
Factor: 2.0
Bit-size: 1.0


100%|██████████| 26/26 [00:00<00:00, 14501.58it/s]


---: 3 : DELTAS 708 (3.792791527563716e-06) :---
Deltas present: 1.0
Enc: 1821
Nor: 2124
Percentage: 0.8573446327683616
Factor: 1.1663920922570017
Bit-size: 2.5720338983050848


100%|██████████| 14/14 [00:00<00:00, 12781.95it/s]


...

---: 22 : DELTAS 135286 (0.0007247338906751199) :---
Deltas present: 0.007027864456176758
Enc: 2762035
Nor: 2976292
Percentage: 0.9280121036511203
Factor: 1.077572152416606
Bit-size: 20.416266280324646


100%|██████████| 721/721 [00:00<00:00, 14462.22it/s]


---: 23 : DELTAS 22982 (0.00012311572724077588) :---
Deltas present: 0.0013194084167480469
Enc: 469563
Nor: 528586
Percentage: 0.8883379431161628
Factor: 1.1256977232022114
Bit-size: 20.431772691671743


100%|██████████| 25/25 [00:00<00:00, 8928.61it/s]


---: 24 : DELTAS 1200 (6.428460216209688e-06) :---
Deltas present: 4.589557647705078e-05
Enc: 24126
Nor: 28800
Percentage: 0.8377083333333334
Factor: 1.1937329022631187
Bit-size: 20.105

---: IN TOTAL :---
Enc: 2457563304
Nor: 2632290904
Percentage: 0.9336214702810826
Factor: 1.0710979040562691
Bit-size: 13.16528994048403
Bit-size no coding: 14.10131446154553
